### Importing Packages 

In [1]:
import requests
import json
import pandas as pd
import psycopg2
import time 
import warnings
import re
import spacy
import numpy as np
from spacy import displacy

### Connecting to the Database  

In [2]:
# Connecting to the newly combined database:
try: 
    conn = psycopg2.connect("host=deep-diver-db.cvulnxnfevj3.us-east-1.rds.amazonaws.com \
    dbname=deep_diver_db user=deep_diver password=Amazon2022")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    
conn.set_session(autocommit=True)

### Get the recipes table

In [3]:
# Get all the table into a dataframe
cur.execute("SELECT * FROM recipes;")
rec_old = pd.DataFrame(cur.fetchall())

In [4]:
# Get the column names and add to the dataframe
cur.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'recipes';")
columns = (cur.fetchall())
col_list =[]
[col_list.append(columns[i][0]) for i in range(len(columns))];
rec_old.columns = col_list

### Define Functions   

In [6]:
# Load the spacy language model, and create the natural language processing (nlp) object.
nlp = spacy.load('en_core_web_sm') 

#### Check the ingredients and their quantities of a specific recipe

In [7]:
# Check the ingredients and their quantities of a specific recipe
def get_ing_list(label):
    """
    This fuctions gets the label of the recipe and returns all of the 
    ingredients and their quantities of the corresponding recipe 
    param: label: string containting the recipe name
    return: dataframe of ingredients and quantities
    """
    sql_ing = """
        SELECT ing_quant from recipes where label = %s
        """
    cur.execute(sql_ing, (label,))
    ings_quant = pd.DataFrame(cur.fetchall()[0][0],columns = ['ing', 'quant'])
    return ings_quant

#### Preprocess the ingredient names

In [8]:
def preprocess(ings):
    """
    This fuctions gets the ingredients and the quantities of the 
    recipe and returns all cleaned and processed versions of the
    ingredient names
    param: ings: dataframe of ingredients and quantities
    return: dataframe of ingredient name pre-processing
    """
    # Convert ingredients column to string and quantity column to integer 
    ings['ing']=ings['ing'].astype(str)
    ings['quant']=ings['quant'].astype(str).astype(int)
    # Clean the data
    ings['ing_c'] = ings['ing'].copy()
    ### Delete all 'in ...'
    ings['ing_c'] = ings['ing_c'].apply(lambda x: re.sub(r"in\s\w*", "", x,flags=re.IGNORECASE).strip())
    ### Delete all 'all...'
    ings['ing_c'] = ings['ing_c'].apply(lambda x: re.sub(r"all(\W|\w)\w*", "", x,flags=re.IGNORECASE).strip())
    ### Convert to lower case
    ings['ing_c'] = ings['ing_c'].apply(lambda x: x.lower())
    # Beetroot check: If beetroot exists turn to 'beets'
    ings['ing_c'] = ings['ing_c'].apply(lambda x: 'beets' if 'beet' in x else x)
    
    search_word = []
    # Apply nlp to each ingredient name and extract Nouns and Pronouns
    for item in ings['ing_c']:
        li=[]; li2=[]
        text = nlp(item)
        for word in text:
            if word.pos_=='NOUN' and str(word).isalpha():
                li.append(str(word)) 
            elif word.pos_=='PROPN' and str(word).isalpha():
                li2.append(str(word))
        ### If no nouns or pronouns exist take the name as itself
        if (not li) and (not li2):
            search_word.append(item)
            ### If just nouns exist:
        elif not li:
            search_word.append(' '.join(li2))    
            ### If just pronouns exist:
        else:
            search_word.append(' '.join(li))
        
    ings['ing_s'] = search_word
    return ings

#### Search the 'Product' table from the database to get the prices for each ingredient

In [9]:
def get_prices(ings):
    """
    This fuctions gets the pre-processed names of the ingredients and 
    returns their prices per 100 grams (or 1 piece if the items that 
    are priced by pieces)
    param: ings: dataframe of ingredient name pre-processing
    return: dataframe of ingredients and their prices per 100 g
    """
    # Get the first entry for the search item
    sql_prod = """
        SELECT *
        FROM 
        (SELECT title, quantity, price, price_per_100
        FROM "Product" 
        WHERE (LOWER(title) LIKE %s AND quantity LIKE %s)
        ORDER BY LENGTH(title) ASC
        FETCH FIRST 20 ROW ONLY
        ) AS temp    
        ORDER BY price_per_100 ASC
        FETCH FIRST 1 ROW ONLY;
        """ 
    prod_det = []
    for i in range(len(ings)):
        ### Get the search text
        search_t = ings.loc[i,'ing_s'] 
        ### Format it for SQL query
        search_text = '%' + search_t + '%'
        ### Define the unit default as gram
        search_unit = '%g%'
        search = (search_text, search_unit)
        
        # Exclude table check:
        # Check whether the search text is within the exclusion table
    
        ex_check = 1 if sum(i in search_t.lower() for i in exclude_table) >= 1 else -1
        ### If on the exclusion list : take the price as 0
        if ex_check == 1:
            prod_det.append((search_t,0,0,0))
        else:
            ### If the item is on the list of products which are priced piecewise
            if search_t in PCE_list:
                ### Change the unit from gram to piece (PCE)
                search_unit = '%PCE%'
                search = (search_text, search_unit)
                cur.execute(sql_prod, search)
            else:
                cur.execute(sql_prod, search)

            try:
                prod_det.append(cur.fetchall()[0])
            except:
                prod_det.append((search_t,0,0,0))

    prod_det = pd.DataFrame(prod_det,columns = ['c_title', 'c_quant', 'c_price','c_pp1'])
    prod_det['c_pp1'] = prod_det['c_pp1'].fillna(0)
    return prod_det

#### Calculate total prices

In [10]:
def total_price(ings_wp):
    """
    This fuctions gets the dataframe of ingredients,quantities and
    their prices per 100 g and returns prices of each ingredient 
    with respect to its quantity
    param: ings_wp: dataframe of ingredients and their prices per 100 g
    return: dataframe of ingredients and their total prices
    """    
    # Calculate prices
    # Create a price column
    ings_wp['price'] = 0
    price_list =[]

    for i in range(len(ings_wp)):
        q = ings_wp['c_quant'][i]
        if q==0:
            price_list.append(0)
        elif 'PCE' in q:
            ### If piecewise prised : take the price for 1 piece
            price_list.append(ings_wp['c_pp1'][i])
        else:
            ### Otherwise calculate the price by using the quantity and 
            ### the price per 100 g
            price_list.append(round(ings_wp['quant'][i]*ings_wp['c_pp1'][i]/100,2))

    ings_wp['Total_price'] = price_list 
    return ings_wp

In [11]:
# Create a list which contains ingredients that are going to be emitted from the
# price calcualtions
exclude_table = ['oil','salt','pepper', 'sugar']
# Create a list which contains ingredients that are priced according to their pieces
PCE_list = ['lemon', 'garlic', 'kiwi', 'egg', 'eggs', 'cucumber', 'cucumbers', 
            'avocado', 'mango', 'pineapple', 'melon', 'watermelon']

#### Run the whole procedure for each recipe

In [ ]:
Prices=[0] * len(rec_old)
for i in range(len(rec_old)):
    label = rec_old['label'][i]
    # Get the ingredients
    ings_quant = get_ing_list(label)
    # Preporcess the ingredients list
    ings = preprocess(ings_quant)
    # Get prices for the ingredients
    prod_det = get_prices(ings)
    # Combine Dataframes ing and product details
    ings_wp = pd.concat( [ings, prod_det], axis=1)
    # Calculate total prices for each ingredient
    ings_wp_n = total_price(ings_wp)
    price = sum(ings_wp_n['Total_price'])
    Prices[i] = price

In [12]:
rec_old['Prices'] = Prices
rec_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39693 entries, 0 to 39692
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   label          39693 non-null  object
 1   search_query   39693 non-null  object
 2   image          39693 non-null  object
 3   url            39693 non-null  object
 4   edamam_url     39693 non-null  object
 5   diets_labels   39693 non-null  object
 6   health_labels  39693 non-null  object
 7   ingredients    39693 non-null  object
 8   ing_quant      39693 non-null  object
 9   meal_type      39693 non-null  object
 10  dish_type      39693 non-null  object
 11  price          39693 non-null  object
dtypes: object(12)
memory usage: 3.6+ MB


In [ ]:
# Load the table to the database
# Create table if not exists   
sql = """
    CREATE TABLE IF NOT EXISTS recipes (
        Label          TEXT PRIMARY KEY,
        Search_Query   TEXT NOT NULL,
        Image          TEXT,
        Url            TEXT,
        Edamam_url     TEXT NOT NULL,
        Diets_Labels   TEXT,
        Health_Labels  TEXT,
        Ingredients    TEXT,
        Ing_Quant      TEXT ARRAY,
        Meal_Type      TEXT,
        Dish_Type      TEXT,
        Price          TEXT) ; """

cur.execute(sql)
conn.commit()

In [ ]:
# Enter the values
sql_in = """
INSERT INTO recipes (Label, Search_Query, Image, Url, Edamam_url, Diets_Labels, Health_Labels, Ingredients, Ing_Quant, Meal_Type, Dish_Type, Price)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
for i in range (len(rec_old)):
    try:
        values = (rec_old.iloc[i][0], rec_old.iloc[i][1], rec_old.iloc[i][2], rec_old.iloc[i][3], 
                  rec_old.iloc[i][4], rec_old.iloc[i][5], rec_old.iloc[i][6], rec_old.iloc[i][7], 
                  rec_old.iloc[i][8], rec_old.iloc[i][9], rec_old.iloc[i][10], rec_old.iloc[i][11])
        cur.execute(sql_in, values)

    except psycopg2.errors.UniqueViolation:
        # In case of duplicate recipes
        pass

### Example Visualization

In [13]:
label = "Beetroot and goat's cheese focaccia"

In [14]:
# Get the ingredients
ings_quant = get_ing_list(label)
ings_quant

,ing,quant
0,yeast,8
1,caster sugar,4
2,water,375
3,flour,450
4,table salt,12
5,olive oil,27
6,beetroot,302
7,Pineapple & Mint juice,81
8,caster sugar,55
9,lemon juice,3


In [15]:
# Preporcess the ingredients list
ings = preprocess(ings_quant)
ings

,ing,quant,ing_c,ing_s
0,yeast,8,yeast,yeast
1,caster sugar,4,caster sugar,caster sugar
2,water,375,water,water
3,flour,450,flour,flour
4,table salt,12,table salt,table salt
5,olive oil,27,olive oil,olive oil
6,beetroot,302,beets,beets
7,Pineapple & Mint juice,81,pineapple & mint juice,pineapple mint juice
8,caster sugar,55,caster sugar,caster sugar
9,lemon juice,3,lemon juice,lemon juice


In [16]:
# Get prices for the ingredients
prod_det = get_prices(ings)
prod_det

,c_title,c_quant,c_price,c_pp1
0,Yeast,42g,0.25,0.60
1,caster sugar,0,0.00,0.00
2,Prix Garantie watermelon,5500 g,9.30,0.17
3,Prix Garantie Braid Bread Flour,1000g,1.50,0.15
4,table salt,0,0.00,0.00
5,olive oil,0,0.00,0.00
6,Naturaplan Organic Raw Red Beets ca. 1kg,1000 g,4.70,0.47
7,pineapple mint juice,0,0.00,0.00
8,caster sugar,0,0.00,0.00
9,Prix Garantie Lemon Juice,207g,0.60,0.29


In [17]:
# Combine Dataframes ing and product details
ings_wp = pd.concat( [ings, prod_det], axis=1)
ings_wp

,ing,quant,ing_c,ing_s,c_title,c_quant,c_price,c_pp1
0,yeast,8,yeast,yeast,Yeast,42g,0.25,0.60
1,caster sugar,4,caster sugar,caster sugar,caster sugar,0,0.00,0.00
2,water,375,water,water,Prix Garantie watermelon,5500 g,9.30,0.17
3,flour,450,flour,flour,Prix Garantie Braid Bread Flour,1000g,1.50,0.15
4,table salt,12,table salt,table salt,table salt,0,0.00,0.00
5,olive oil,27,olive oil,olive oil,olive oil,0,0.00,0.00
6,beetroot,302,beets,beets,Naturaplan Organic Raw Red Beets ca. 1kg,1000 g,4.70,0.47
7,Pineapple & Mint juice,81,pineapple & mint juice,pineapple mint juice,pineapple mint juice,0,0.00,0.00
8,caster sugar,55,caster sugar,caster sugar,caster sugar,0,0.00,0.00
9,lemon juice,3,lemon juice,lemon juice,Prix Garantie Lemon Juice,207g,0.60,0.29


In [18]:
# Calculate total prices for each ingredient
ings_wp_n = total_price(ings_wp)
ings_wp_n

,ing,quant,ing_c,ing_s,c_title,c_quant,c_price,c_pp1,price,Total_price
0,yeast,8,yeast,yeast,Yeast,42g,0.25,0.60,0,0.05
1,caster sugar,4,caster sugar,caster sugar,caster sugar,0,0.00,0.00,0,0.00
2,water,375,water,water,Prix Garantie watermelon,5500 g,9.30,0.17,0,0.64
3,flour,450,flour,flour,Prix Garantie Braid Bread Flour,1000g,1.50,0.15,0,0.68
4,table salt,12,table salt,table salt,table salt,0,0.00,0.00,0,0.00
5,olive oil,27,olive oil,olive oil,olive oil,0,0.00,0.00,0,0.00
6,beetroot,302,beets,beets,Naturaplan Organic Raw Red Beets ca. 1kg,1000 g,4.70,0.47,0,1.42
7,Pineapple & Mint juice,81,pineapple & mint juice,pineapple mint juice,pineapple mint juice,0,0.00,0.00,0,0.00
8,caster sugar,55,caster sugar,caster sugar,caster sugar,0,0.00,0.00,0,0.00
9,lemon juice,3,lemon juice,lemon juice,Prix Garantie Lemon Juice,207g,0.60,0.29,0,0.01


In [19]:
price = round(sum(ings_wp_n['Total_price']))
price

4